# Cargar y enriquecer varios tipos de archivos Azure AI Search

En este Jupyter Notebook, creamos y ejecutamos pasos de enriquecimiento para desbloquear contenido buscable en el blob de Azure especificado. Realiza operaciones sobre contenido mixto en Azure Storage, como imágenes y archivos de aplicaciones, utilizando un conjunto de habilidades que analiza y extrae información de texto que se convierte en buscable en Azure Cognitive Search. 
La muestra de referencia se puede encontrar en [Tutorial: Use Python and AI to generate searchable content from Azure blobs](https://docs.microsoft.com/azure/search/cognitive-search-tutorial-blob-python).

En esta demostración vamos a utilizar un contenedor Blob Storage privado (para que podamos imitar un escenario de datalake privado) que tiene ~9.8k PDFs de publicaciones de Ciencias de la Computación del conjunto de datos Arxiv.
https://www.kaggle.com/datasets/Cornell-University/arxiv
Si desea explorar el conjunto de datos, vaya [AQUÍ](https://console.cloud.google.com/storage/browser/arxiv-dataset/arxiv/cs/pdf?pageState=(%22StorageObjectListTable%22:(%22f%22:%22%255B%255D%22))&prefix=&forceOnObjectsSortingFiltering=false)<br>
Nota: Este conjunto de datos se ha copiado en un contenedor blob azure público para esta demostración

Aunque aquí sólo se utilizan archivos PDF, esto se puede hacer a una escala mucho mayor y Azure Cognitive Search admite una serie de otros formatos de archivo, incluyendo: Microsoft Office (DOCX/DOC, XSLX/XLS, PPTX/PPT, MSG), HTML, XML, ZIP y archivos de texto sin formato (incluido JSON).
Azure Search admite las siguientes fuentes: [Galería de fuentes de datos](https://learn.microsoft.com/EN-US/AZURE/search/search-data-sources-gallery)
Este cuaderno crea los siguientes objetos en su servicio de búsqueda:
+ fuente de datos (data source)
+ índice de búsqueda (search index)
+ conjunto de habilidades (skillset)
+ indexador (indexer)

Este cuaderno llama a las [Search REST APIs](https://docs.microsoft.com/rest/api/searchservice/), pero también puedes utilizar la biblioteca cliente Azure.Search.Documents en el SDK de Azure para Python para realizar los mismos pasos. Consulta este [Python quickstart](https://docs.microsoft.com/azure/search/search-get-started-python) para más detalles.
Para ejecutar este cuaderno, ya deberías haber creado los servicios Azure en README. Una vez hecho esto, puedes ejecutar todas las celdas, pero la consulta no devolverá resultados hasta que el indexador haya terminado y el índice de búsqueda esté cargado. 
Recomendamos ejecutar cada paso y asegurarse de que se completa antes de continuar.

![cog-search](./images/Cog-Search-Enrich.png)

In [22]:
import os
import json
import requests
from dotenv import load_dotenv
load_dotenv("credentials.env")

# Name of the container in your Blob Storage Datasource ( in credentials.env)
BLOB_CONTAINER_NAME = "cybersecurity"

In [23]:
# Define the names for the data source, skillset, index and indexer
datasource_name = "cogsrch-datasource-kiografia"
index_name = "cogsrch-index-kiografia"
skillset_name = "cogsrch-skillset-kiografia"
indexer_name = "cogsrch-indexer-kiografia"

In [24]:
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

## Create Data Source (Blob container with the Arxiv CS pdfs)

In [25]:
# The following code sends the json paylod to Azure Search engine to create the Datasource

datasource_payload = {
    "name": datasource_name,
    "description": "Demo files to demonstrate cognitive search capabilities.",
    "type": "azureblob",
    "credentials": {
        "connectionString": os.environ['BLOB_CONNECTION_STRING']
    },
    "dataDeletionDetectionPolicy" : {
        "@odata.type" :"#Microsoft.Azure.Search.NativeBlobSoftDeleteDeletionDetectionPolicy" # this makes sure that if the item is deleted from the source, it will be deleted from the index
    },
    "container": {
        "name": BLOB_CONTAINER_NAME
    }
}
r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/datasources/" + datasource_name,
                 data=json.dumps(datasource_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)
print(os.environ['BLOB_CONNECTION_STRING'])

204
True


- 201 - Creado con éxito
- 204 - Sobrescrito correctamente
- 40X - Error de autenticación
Para obtener información sobre la detección de archivos modificados y eliminados, consulte [AQUÍ](https://learn.microsoft.com/en-us/azure/search/search-howto-index-changed-deleted-blobs?tabs=rest-api)

In [ ]:
# If you have a 403 code, probably you have a wrong endpoint or key, you can debug by uncomment this
# r.text

## Crear índice

En Azure AI Search, un índice de búsqueda es su contenido de búsqueda, disponible para el motor de búsqueda para indexación, búsqueda de texto completo, búsqueda vectorial, búsqueda híbrida y consultas filtradas. Un índice se define mediante un esquema y se guarda en el servicio de búsqueda, con la importación de datos como segundo paso. Este contenido existe dentro de su servicio de búsqueda, aparte de sus almacenes de datos primarios, lo cual es necesario para los tiempos de respuesta de milisegundos que se esperan en las aplicaciones de búsqueda modernas. Excepto en los escenarios de indexación impulsada por indexadores, el servicio de búsqueda nunca se conecta a los datos de origen ni los consulta.

Referencia:
https://learn.microsoft.com/en-us/azure/search/search-what-is-an-index

Observa a continuación cómo estamos creando un almacén vectorial. En Azure AI Search, un almacén de vectores tiene un esquema de índice que define los campos vectoriales y no vectoriales, una configuración de vectores para los algoritmos que crean el espacio de incrustación y ajustes en las definiciones de campos vectoriales que se utilizan en las solicitudes de consulta. 

También establecemos una clasificación semántica sobre un conjunto de resultados, promoviendo los resultados semánticamente más relevantes a la parte superior de la pila. También se pueden obtener subtítulos semánticos, con resaltados sobre los términos y frases más relevantes, y respuestas semánticas.

In [26]:
# Create an index
# Queries operate over the searchable fields and filterable fields in the index
index_payload = {
    "name": index_name,
    "vectorSearch": {
        "algorithms": [
            {
                "name": "myalgo",
                "kind": "hnsw"
            }
        ],
        "vectorizers": [
            {
                "name": "openai",
                "kind": "azureOpenAI",
                "azureOpenAIParameters":
                {
                    "resourceUri" : os.environ['AZURE_OPENAI_ENDPOINT'],
                    "apiKey" : os.environ['AZURE_OPENAI_API_KEY'],
                    "deploymentId" : os.environ['EMBEDDING_DEPLOYMENT_NAME']
                }
            }
        ],
        "profiles": [
            {
                "name": "myprofile",
                "algorithm": "myalgo",
                "vectorizer":"openai"
            }
        ]
    },
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": {
                        "fieldName": "title"
                    },
                    "prioritizedContentFields": [
                        {
                            "fieldName": "chunk"
                        }
                    ],
                    "prioritizedKeywordsFields": []
                }
            }
        ]
    },
    "fields": [
        {"name": "id", "type": "Edm.String", "key": "true", "analyzer": "keyword", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false","facetable": "false"},
        {"name": "ParentKey", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "true", "sortable": "false"},
        {"name": "title", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "true", "sortable": "false"},
        {"name": "name", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "location", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},   
        {"name": "chunk","type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {
            "name": "chunkVector",
            "type": "Collection(Edm.Single)",
            "dimensions": 1536, # IMPORTANT: Make sure these dimmensions match your embedding model name
            "vectorSearchProfile": "myprofile",
            "searchable": "true",
            "retrievable": "true",
            "filterable": "false",
            "sortable": "false",
            "facetable": "false"
        }
    ]
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

204
True


In [ ]:
# Uncomment if you find an error
# r.text

#### Capacidades de búsqueda semántica
Como puede ver arriba, en la carga útil del índice hay una "configuración semántica". ¿En qué consiste?

La clasificación semántica es un conjunto de capacidades relacionadas con las consultas que mejoran la calidad de un resultado de búsqueda inicial clasificado por BM25 o RRF para consultas basadas en texto. Cuando se activa en el servicio de búsqueda, la clasificación semántica amplía el proceso de ejecución de consultas de dos maneras:

    En primer lugar, añade una clasificación secundaria a un conjunto de resultados inicial que se ha clasificado mediante BM25 o RRF. Esta clasificación secundaria utiliza modelos multilingües de aprendizaje profundo adaptados de Microsoft Bing para promover los resultados semánticamente más relevantes.
    
    En segundo lugar, extrae y devuelve subtítulos y respuestas en la respuesta, que puede renderizar en una página de búsqueda para mejorar la experiencia de búsqueda del usuario.

Para una explicación más detallada y conocer las limitaciones, consulte [AQUÍ](https://learn.microsoft.com/en-us/azure/search/semantic-ranking)

## Crear Skillset - OCR, Text Splitter, AzureOpenAIEmbeddingSkill
Ahora tenemos que crear el conjunto de habilidades. Se trata de un conjunto de pasos en los que utilizamos AI Services para enriquecer los documentos extrayendo información, aplicando OCR, dividiendo e incrustando trozos, entre otras habilidades.

https://learn.microsoft.com/en-us/azure/search/cognitive-search-working-with-skillsets

https://learn.microsoft.com/en-us/azure/search/cognitive-search-predefined-skills

Observe a continuación que estamos utilizando IndexProjections. Por defecto, un documento procesado dentro de un conjunto de habilidades se asigna a un único documento en el índice de búsqueda. Esto significa que si realizas un chunking de un texto de entrada y luego realizas enriquecimientos en cada chunk, el resultado en el índice cuando se mapea a través de outputFieldMappings es un array de los enriquecimientos generados. **Con las proyecciones de índice, se define un contexto en el que asignar cada fragmento de datos enriquecidos a su propio documento de búsqueda**. Esto permite aplicar una correspondencia uno a muchos de los datos enriquecidos de un documento al índice de búsqueda.
    
El parámetro: `"projectionMode": "skipIndexingParentDocuments"` nos permite omitir la indexación de los documentos padre, y mantener sólo el índice con los chunks y sus vectores.

### Ciclo de vida del contenido
Si el origen de datos del indizador admite el seguimiento de cambios y la detección de borrados, el proceso de indización puede sincronizar los índices primario (documentos parend) y secundario (chunks) para recoger esos cambios.
Cada vez que se ejecuta el indizador y el conjunto de capacidades, las proyecciones del índice se actualizan si el conjunto de capacidades o los datos de origen subyacentes han cambiado. Cualquier cambio recogido por el indexador se propaga a través del proceso de enriquecimiento a las proyecciones del índice, garantizando que los datos proyectados sean una representación actual del contenido de la fuente de datos de origen. Esto le ahorrará semanas de programación y muchos quebraderos de cabeza intentando mantener el contenido sincronizado.

In [27]:
# Create a skillset
skillset_payload = {
    "name": skillset_name,
    "description": "e2e Skillset for RAG - Files",
    "skills":
    [
        {
            "@odata.type": "#Microsoft.Skills.Vision.OcrSkill",
            "description": "Extract text (plain and structured) from image.",
            "context": "/document/normalized_images/*",
            "defaultLanguageCode": "en",
            "detectOrientation": True,
            "inputs": [
                {
                  "name": "image",
                  "source": "/document/normalized_images/*"
                }
            ],
                "outputs": [
                {
                  "name": "text",
                  "targetName" : "images_text"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.MergeSkill",
            "description": "Create merged_text, which includes all the textual representation of each image inserted at the right location in the content field. This is useful for PDF and other file formats that supported embedded images.",
            "context": "/document",
            "insertPreTag": " ",
            "insertPostTag": " ",
            "inputs": [
                {
                  "name":"text", "source": "/document/content"
                },
                {
                  "name": "itemsToInsert", "source": "/document/normalized_images/*/images_text"
                },
                {
                  "name":"offsets", "source": "/document/normalized_images/*/contentOffset"
                }
            ],
            "outputs": [
                {
                  "name": "mergedText", 
                  "targetName" : "merged_text"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.SplitSkill",
            "context": "/document",
            "textSplitMode": "pages",  # although it says "pages" it actally means chunks, not actual pages
            "maximumPageLength": 5000, # 5000 characters is default and a good choice
            "pageOverlapLength": 750,  # 15% overlap among chunks
            "defaultLanguageCode": "en",
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/merged_text"
                }
            ],
            "outputs": [
                {
                    "name": "textItems",
                    "targetName": "chunks"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.AzureOpenAIEmbeddingSkill",
            "description": "Azure OpenAI Embedding Skill",
            "context": "/document/chunks/*",
            "resourceUri": os.environ['AZURE_OPENAI_ENDPOINT'],
            "apiKey": os.environ['AZURE_OPENAI_API_KEY'],
            "deploymentId": os.environ['EMBEDDING_DEPLOYMENT_NAME'],
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/chunks/*"
                }
            ],
            "outputs": [
                {
                    "name": "embedding",
                    "targetName": "vector"
                }
            ]
        }
    ],
    "indexProjections": {
        "selectors": [
            {
                "targetIndexName": index_name,
                "parentKeyFieldName": "ParentKey",
                "sourceContext": "/document/chunks/*",
                "mappings": [
                    {
                        "name": "title",
                        "source": "/document/title"
                    },
                    {
                        "name": "name",
                        "source": "/document/name"
                    },
                    {
                        "name": "location",
                        "source": "/document/location"
                    },
                    {
                        "name": "chunk",
                        "source": "/document/chunks/*"
                    },
                    {
                        "name": "chunkVector",
                        "source": "/document/chunks/*/vector"
                    }
                ]
            }
        ],
        "parameters": {
            "projectionMode": "skipIndexingParentDocuments"
        }
    },
    "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": os.environ['COG_SERVICES_NAME'],
        "key": os.environ['COG_SERVICES_KEY']
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/skillsets/" + skillset_name,
                 data=json.dumps(skillset_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

204
True


In [ ]:
# Uncomment if you find an error
# print(r.text)

## Crear y ejecutar el Indexador - (ejecuta el pipeline)

Los tres componentes que has creado hasta ahora (fuente de datos, conjunto de habilidades, índice) son entradas para un indexador. La creación del indexador en Azure Cognitive Search es el evento que pone en marcha todo el proceso.

In [28]:
# Create an indexer
indexer_payload = {
    "name": indexer_name,
    "dataSourceName": datasource_name,
    "targetIndexName": index_name,
    "skillsetName": skillset_name,
    #"schedule" : { "interval" : "PT30M"}, # How often do you want to check for new content in the data source
    "fieldMappings": [
        {
          "sourceFieldName" : "metadata_title",
          "targetFieldName" : "title"
        },
        {
          "sourceFieldName" : "metadata_storage_name",
          "targetFieldName" : "name"
        },
        {
          "sourceFieldName" : "metadata_storage_path",
          "targetFieldName" : "location"
        }
    ],
    "outputFieldMappings":[],
    "parameters":
    {
        "maxFailedItems": -1,
        "maxFailedItemsPerBatch": -1,
        "configuration":
        {
            "dataToExtract": "contentAndMetadata",
            "imageAction": "generateNormalizedImages"
        }
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name,
                 data=json.dumps(indexer_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

201
True


In [29]:
# Uncomment if you find an error
#r.text
print(r.text)

{"@odata.context":"https://azure-search-dorak.search.windows.net/$metadata#indexers/$entity","@odata.etag":"\"0x8DC9AE9E91B5FEE\"","name":"cogsrch-indexer-kiografia","description":null,"dataSourceName":"cogsrch-datasource-kiografia","skillsetName":"cogsrch-skillset-kiografia","targetIndexName":"cogsrch-index-kiografia","disabled":null,"schedule":null,"parameters":{"batchSize":null,"maxFailedItems":-1,"maxFailedItemsPerBatch":-1,"base64EncodeKeys":null,"configuration":{"dataToExtract":"contentAndMetadata","imageAction":"generateNormalizedImages"}},"fieldMappings":[{"sourceFieldName":"metadata_title","targetFieldName":"title","mappingFunction":null},{"sourceFieldName":"metadata_storage_name","targetFieldName":"name","mappingFunction":null},{"sourceFieldName":"metadata_storage_path","targetFieldName":"location","mappingFunction":null}],"outputFieldMappings":[],"cache":null,"encryptionKey":null}


In [30]:
# Optionally, get indexer status to confirm that it's running
try:
    r = requests.get(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name +
                     "/status", headers=headers, params=params)
    # pprint(json.dumps(r.json(), indent=1))
    print(r.status_code)
    print("Status:",r.json().get('lastResult').get('status'))
    print("Items Processed:",r.json().get('lastResult').get('itemsProcessed'))
    print(r.ok)
    
except Exception as e:
    print("Wait a few seconds until the process starts and run this cell again.")

200
Status: success
Items Processed: 3
True


# Referencias

- https://learn.microsoft.com/en-us/azure/search/cognitive-search-tutorial-blob
- https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/search
- https://learn.microsoft.com/en-us/azure/search/search-get-started-vector